In [ ]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tfsm
import torchvision.models as models

from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm, trange

In [ ]:
def get_images_df(IMG_DIR):
    df_list = []
    for class_dir in list(IMG_DIR.iterdir()):
        class_name = class_dir.name
        all_images = list(class_dir.iterdir())
        for img in all_images:
            df_list.append([str(img), class_name])
    df = pd.DataFrame(df_list, columns=["img_path","class"])
    return df

In [ ]:
BASE_DIR = Path('/kaggle/input/100-bird-species')

VALID_DIR = BASE_DIR/'valid'
TRAIN_DIR = BASE_DIR/'train'
TEST_DIR = BASE_DIR/'test'

In [ ]:
train_df = get_images_df(TRAIN_DIR)
valid_df = get_images_df(VALID_DIR)
test_df = get_images_df(TEST_DIR)

class2idx = {x:i for i, x in enumerate(list(train_df['class'].unique()))}


The Bird Dataset

In [ ]:
class BirdDataset(Dataset):
    
    def __init__(self, images_df, class2idx, transforms = None):
        
        super().__init__()
        self.images_df = images_df
        self.class2idx = class2idx
        self.transforms = transforms
        self.idx2class = {v:k for k, v in class2idx.items()}
        
    def __getitem__(self, index):
    
        image_path = self.images_df.iloc[index]['img_path']
        image_class = self.images_df.iloc[index]['class']
        
        #Reading image
        image = Image.open(image_path)

        label = self.class2idx[image_class]
        
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image, label
        
        
        
    def __len__(self):
        return len(self.images_df)

In [ ]:
def get_transform(train=False):
    if train:
        transforms = [tfsm.ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4),
                      tfsm.RandomHorizontalFlip(),
                      tfsm.RandomVerticalFlip(),
                      tfsm.RandomRotation(2.8),
                      tfsm.ToTensor(),
                      tfsm.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                      ]
    else:
        transforms = [tfsm.ToTensor(),
                      tfsm.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                      ]
        
    return tfsm.Compose(transforms)

In [ ]:
# from https://www.kaggle.com/jainamshah17/pytorch-starter-image-classification
def calc_accuracy(true,pred):
    pred = F.softmax(pred, dim = 1)
    true = torch.zeros(pred.shape[0], pred.shape[1]).scatter_(1, true.unsqueeze(1), 1.)
    acc = (true.argmax(-1) == pred.argmax(-1)).float().detach().numpy()
    acc = float((100 * acc.sum()) / len(acc))
    return round(acc, 4)

In [ ]:
def shared_step(batch, model, device, optimizer, criterion, phase='Train'):
    images , labels = batch
    images , labels = images.to(device) , labels.to(device)
    if phase=='Train':
        preds = model(images)
    else:
        with torch.no_grad():
            preds = model(images)
    loss = criterion(preds, labels)

    if phase=='Train':
        loss.backward()
        optimizer.step()
        
    acc = calc_accuracy(labels.cpu(), preds.cpu())
    loss_value = loss.item()
    
    return loss_value, acc

In [ ]:
def one_epoch(model, train_dl, valid_dl, device, optimizer, criterion, print_every=500):
    #Epoch Loss & Accuracy
    train_epoch_loss = []
    train_epoch_accuracy = []
    _iter = 1
    
    #Val Loss & Accuracy
    val_epoch_loss = []
    val_epoch_accuracy = []
    
    # training phase
    model.train()
    for batch in train_dl:
        train_loss, train_acc =  shared_step(batch, model, device, optimizer, criterion, phase='Train')
        
        train_epoch_loss.append(train_loss)
        train_epoch_accuracy.append(train_acc)
        if _iter % print_every == 0 and print_every !=-1:
            print("> Iteration {} < ".format(_iter), end="")
            print(" Iter Loss = {}".format(round(train_loss, 4)), end="")
            print(" Iter Accuracy = {} % \n".format(train_acc))
        
        _iter += 1

    # validation phase
    model.eval()
    for batch in valid_dl:
        valid_loss, valid_acc =  shared_step(batch, model, device, optimizer, criterion, phase='valid')
        val_epoch_loss.append(valid_loss)
        val_epoch_accuracy.append(valid_acc)
    
    train_epoch_loss = np.mean(train_epoch_loss)
    train_epoch_accuracy = np.mean(train_epoch_accuracy)
    
    val_epoch_loss = np.mean(val_epoch_loss)
    val_epoch_accuracy = np.mean(val_epoch_accuracy)
    
    return {"train_loss":train_epoch_loss,
            "train_acc":train_epoch_accuracy,
            "val_loss":val_epoch_loss,
            "val_acc":val_epoch_accuracy
            }
        

In [ ]:
def train(model, train_dl, valid_dl, device, optimizer, criterion, n_epochs = 200, print_every=500):
    best_val_loss = 1e10
    train_acc = []
    train_loss = []
    val_acc = []
    val_loss = []
    t = trange(1,n_epochs+1, desc='Epoch ', leave=True, ncols=800)
    with tqdm(bar_format='{desc}{bar}' , ncols=800) as line1:
        for epoch in t:
            result = one_epoch(model, train_dl, valid_dl, device, optimizer, criterion,print_every=print_every)
            train_acc.append(result["train_acc"])
            train_loss.append(result["train_loss"])
            val_acc.append(result["val_acc"])
            val_loss.append(result["val_loss"])
            status = f"Epoch {epoch}/{n_epochs}: train_acc = {result['train_acc']:0.4}% "
            status = status + f"train_loss = {result['train_loss']:0.4} "
            status = status + f"val_acc = {result['val_acc']:0.4}% "
            status = status + f"val_loss = {result['val_loss']:0.4} "
            t.set_description(status)
            t.refresh()
            if result["val_loss"] < best_val_loss:
                if epoch == 1:
                    l1 = f"Epoch {epoch}: Validation loss {result['val_loss']:0.4} saving the model"
                else:
                    l1 = f"Epoch {epoch}: found better validation loss was {best_val_loss:0.4}, now {result['val_loss']:0.4}"
                    
                best_val_loss = result["val_loss"]
                torch.save(model.state_dict(), "bestVal.pt")
                line1.set_description(l1)
        
        train_history = {"train_acc":train_acc,
                         "train_loss":train_loss,
                         "val_acc":val_acc,
                         "val_loss":val_loss
                        }
    return train_history

In [ ]:
def get_model(model_name, num_classes=255, pretrained=True):
    defined = ["densenet121", "resnet18", "resnet34", "resnext50"]
    try:
        assert model_name in defined
    except:
        print("Model is not defined use one of\n", defined)
        return
    if model_name == "densenet121":
        model = models.densenet121(pretrained = pretrained)
        model.classifier = nn.Sequential(
            nn.Linear(1024, 4096, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(4096, 2048, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(2048, num_classes)
        );
        
    if model_name == "resnet18":
        model = models.resnet18(pretrained = pretrained)
        model.fc = nn.Sequential(
            nn.Linear(512, 1024, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(1024, 2048, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(2048, num_classes)
        );
        
        
    if model_name == "resnet34":
        model = models.resnet34(pretrained = pretrained)
        model.fc = nn.Sequential(
            nn.Linear(512, 1024, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(1024, 2048, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(2048, num_classes)
        );
    
    if model_name == "resnext50":
        model = models.resnext50_32x4d(pretrained = pretrained)
        model.fc = nn.Sequential(
            nn.Linear(2048, 4096, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(4096, 2048, bias = True),
            nn.ReLU(inplace = True),
            nn.Dropout(0.4),
            nn.Linear(2048, num_classes)
        );
        
    return model 

In [ ]:
train_ds = BirdDataset(train_df,class2idx, transforms=get_transform(train=True))
valid_ds = BirdDataset(valid_df,class2idx, transforms=get_transform(train=False))
test_ds = BirdDataset(test_df,class2idx, transforms=get_transform(train=False))

img, label = train_ds[20]
plt.imshow(img.permute(1,2,0).numpy())


In [ ]:
batch_size = 256
train_dl = DataLoader(
    dataset = train_ds,
    batch_size = batch_size,
    shuffle=True,
    num_workers = 2,
)

valid_dl = DataLoader(
    dataset = valid_ds,
    batch_size = batch_size,
    shuffle=False,
    num_workers = 2,
)

test_dl = DataLoader(
    dataset = test_ds,
    batch_size = batch_size,
    shuffle=False,
    num_workers = 2,
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()

In [ ]:
num_classes = len(class2idx)

In [ ]:
model = get_model("resnet34", num_classes, pretrained=True)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.75)
criterion = nn.CrossEntropyLoss()
model.to(device);

In [ ]:
#n_print = int((len(train_ds)//batch_size)/2)
train_history = train(model, train_dl, valid_dl, device, optimizer, criterion, n_epochs =50, print_every=-1)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(train_history['train_loss'], label="Training Loss")
ax.plot(train_history['val_loss'], label="validation Loss")
plt.xticks(np.arange(0, len(train_history['train_loss'])+1, 10))
ax.legend(loc='upper right',bbox_to_anchor=(1.01, 1.09));

In [ ]:
test_losses = []
test_accuracy = []
model.load_state_dict(torch.load('bestVal.pt'))
model.eval()
for batch in test_dl:
    test_loss, test_acc =  shared_step(batch, model, device, optimizer, criterion, phase='test')
    test_losses.append(test_loss)
    test_accuracy.append(test_acc)
    
avg_test_loss = np.mean(test_loss)
avg_test_accuracy = np.mean(test_accuracy)

In [ ]:
print(f"Average testing accuracy = {avg_test_accuracy:.3}%, average testing loss = {avg_test_loss:.3}")